# SQuadGen + uxarray Mesh Full Demo

This notebook demonstrates how to:

- Clone and build [SQuadGen](https://github.com/MPAS-Dev/SQuadGen)
- Generate a regionally refined mesh using SQuadGen (from within the notebook!)
- Visualize the mesh using [uxarray](https://github.com/UXARRAY/uxarray)

---

## Mesh Generation Steps

```sh
# 1. Clone and build SQuadGen
git clone https://github.com/MPAS-Dev/SQuadGen.git
cd SQuadGen/src
make

# 2. Add SQuadGen to your PATH (so you can call it from anywhere)
export PATH="$PATH:/Users/mbook/squadgen"

# 3. Generate a regionally refined mesh (from notebook or terminal)
SQuadGen --grid_type CS --refine_type LOWCONN --refine_level 2 --resolution 10 --output output_mesh.nc --refine_rect "-110.0,35.0,-105.0,40.0,2"
```

---

## SQuadGen Command Line Options Explained

- `--grid_type CS`: Use a Cubed Sphere grid. Other options: ICO (Icosahedral), OCT1, OCT2.
- `--refine_type LOWCONN`: Use the LOWCONN refinement scheme. Other options: CUBIT, LOWCONNOLD.
- `--refine_level 2`: Number of refinement levels to apply. Higher means more local refinement.
- `--resolution 10`: Base mesh resolution (number of cells along one edge of a panel).
- `--output output_mesh.nc`: Output filename for the generated mesh (NetCDF format).
- `--refine_rect "-110.0,35.0,-105.0,40.0,2"`: Define a rectangular region to refine.
  - Format: `lon_min,lat_min,lon_max,lat_max,refine_level`
  - Here: longitudes -110 to -105, latitudes 35 to 40, refinement level 2.

You can adjust these parameters to focus refinement on other regions or increase mesh density.


In [24]:
import os

# Prepend the SQuadGen directory to PATH for this notebook session
os.environ["PATH"] = "/Users/mbook/SQuadGen:" + os.environ["PATH"]

In [27]:
!which SQuadGen

/Users/mbook/SQuadGen/SQuadGen


In [28]:
import subprocess

mesh_file = 'output_mesh.nc'

# Only generate if not present
if not os.path.exists(mesh_file):
    cmd = [
        'squadgen',
        '--grid_type', 'CS',
        '--refine_type', 'LOWCONN',
        '--refine_level', '2',
        '--resolution', '10',
        '--output', mesh_file,
        '--refine_rect', '-110.0,35.0,-105.0,40.0,2'
    ]
    print('Generating regionally refined mesh with SQuadGen...')
    subprocess.run(cmd, check=True)
else:
    print('Mesh file already exists, skipping generation.')

Generating regionally refined mesh with SQuadGen...
Parameters:
  --grid_type <string> ["CS"] (Options: CS | ICO | OCT1 | OCT2)
  --refine_type <string> ["LOWCONN"] (Options: LOWCONN | CUBIT | LOWCONNOLD)
  --refine_level <integer> [2] 
  --resolution <integer> [10] 
  --refine_file <string> [""] 
  --refine_rect <string> ["-110.0,35.0,-105.0,40.0,2"] 
  --output <string> ["output_mesh.nc"] 
  --loadcsrefinementmap <bool> [false] 
  --smooth_type <string> ["NONE"] (Options: NONE | SPRING | PRESSURE)
  --smooth_dist <integer> [1] (Smooth distance, -1 = smooth entire mesh)
  --smooth_iter <integer> [10] 
  --lon_base <double> [-180.000000] 
  --lat_base <double> [0.000000] 
  --x_rotate <double> [0.000000] 
  --y_rotate <double> [0.000000] 
  --lon_ref <double> [0.000000] 
  --lat_ref <double> [0.000000] 
  --orient_ref <double> [0.000000] 
  --tessellate <integer> [0] 
  --subcellres <integer> [0] 
  --invert <bool> [false] 
  --block_refine <bool> [false] 
-----------------------------

---

## Visualize the Mesh with uxarray

Below, we load the generated mesh and plot it using uxarray. You can further analyze or visualize the mesh as needed.


In [32]:
import uxarray as ux
import matplotlib.pyplot as plt

mesh_file = 'output_mesh.nc'

if not os.path.exists(mesh_file):
    print(f'Mesh file not found: {mesh_file}')
else:
    mesh = ux.open_dataset(mesh_file, mesh_file)

mesh.uxgrid

<uxarray.Grid>
Original Grid Type: Exodus
Grid Dimensions:
  * n_node: 838
  * n_face: 836
  * n_max_face_nodes: 4
Grid Coordinates (Spherical):
  * node_lon: (838,)
  * node_lat: (838,)
Grid Coordinates (Cartesian):
  * node_x: (838,)
  * node_y: (838,)
  * node_z: (838,)
Grid Connectivity Variables:
  * face_node_connectivity: (836, 4)
Grid Descriptor Variables:

In [33]:
mesh.uxgrid.plot()

:Path   [Longitude,Latitude]